## Reto 2 - Orquestacion de cadenas de generacion de código

Para llevar a cabo la orquestacion de las diferentes estapas de la generacion asisteida del proyecto, se inicio con la creación y prueba de cada una de ellas, para determinar el flujo de cada cadena de generación por etapa y llevar acabo pruebas con el fin de determinar las dependencias en datos y resultados entre cada una de ellas.
Como antecedentes se usaron los resultados obtenidos en la etapa de desarrollo de la arquitectura del proyecto, que se presento en el notebook Reto_2_LLM_Diseño_Arquitectura.ipyn.
Las etapas que se desarrollaron son:
- Generacion de melos Pydantic a partir de la configuración de la estructura de los datos para los usuarios e incidentes.
- Generación del backend.
- Generación del frontend.
- Generación del stack de pruebas para el backend
- Generacion del stack de pruebas para el frontend
- Generacion de los archivos para deployment de infraestructura (base de datos, backend, frontend, etc)

Una vez que se ceraron y se comprobaron la correcta ejecución de cada etapa, se generaron las funciones de orquestación de cada uno de los compoenentes y etapas de la app.

## Parte 1. Setup del entorno

In [7]:
# Importamos las librerias
import os
import yaml
import json
import ast
import time
import logging
from pathlib import Path
from typing import Dict, List, Optional, Any, Literal
from dataclasses import dataclass
from datetime import datetime
from jinja2 import Template

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.callbacks import BaseCallbackHandler
from pydantic import BaseModel, Field, EmailStr


In [113]:
# Inicializamos el modelo.
from dotenv import load_dotenv
load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  Configura OPENAI_API_KEY en tu archivo .env")
else:
    print("✅ OpenAI API Key configurada")

# Configurar modelo
model = ChatOpenAI(model="gpt-5", temperature=0)
print("🤖 Modelo listo")

✅ OpenAI API Key configurada
🤖 Modelo listo


## Parte 2: Modelos de configuracion Pydantic para las estructuras de datos a usar

In [34]:
from typing import Literal, Union

# Modelos de configuración

class FieldConfig(BaseModel):
    name: str
    type: Literal["int", "float", "str", "bool", "datetime","EmailStr"]
    description: str = ""
    required: bool = False
    constraints: Optional[Dict[str, Any]] = None

class ResourceConfig(BaseModel):
    resource_name: str
    class_name: str
    fields: List[FieldConfig]
    auth_required: bool = False
    cache_enabled: bool = False
    pagination: bool = True
    soft_delete: bool = False

In [35]:
# Creamos la configuración del usuario que se va a administrar en la app
user_config = ResourceConfig(
    resource_name="user",
    class_name="user", 
    fields=[
        FieldConfig(name="id", type="str", description="Identificacdor del usuario", 
                   constraints={"min_length": 1, "max_length": 50}),
        FieldConfig(name="email", type="EmailStr", description="email del usuario",
                   constraints={"min_length": 1, "max_length": 100}),
        FieldConfig(name="hashed_password", type="str", required=True, description="Password del usuario",
                    constraints={"min_length": 1, "max_length": 20}),
        FieldConfig(name="fullname", type="str", required=True, description="Password del usuario",
                    constraints={"min_length": 1, "max_length": 200}),
        FieldConfig(name="role", type="str", required=True, description="Rol del usuario",
                    constraints={"min_length": 1, "max_length": 50}),
        FieldConfig(name="created_at", type="datetime", required=True, description="Rol del usuario")
    ],
    auth_required=True,
    cache_enabled=True,
    pagination=True,
    soft_delete=True
)
# Creamos la configuración del incidente que va almanecarse con la app
incident_config = ResourceConfig(
    resource_name="incident",
    class_name="incident", 
    fields=[
        FieldConfig(name="id", type="str", description="Identificacdor del incidente", 
                   constraints={"min_length": 1, "max_length": 50}),
        FieldConfig(name="title", type="str", description="titulo del incidente",
                   constraints={"min_length": 1, "max_length": 100}),
        FieldConfig(name="descripction", type="str", required=True, description="Descripcion del incidente",
                    constraints={"min_length": 1, "max_length": 20}),
        FieldConfig(name="severity", type="int", required=True, description="Severidad del incidente",
                    constraints={"ge": 0, "le": 10}),
        FieldConfig(name="created_by", type="str", required=True, description="Rol del usuario",
                    constraints={"min_length": 1, "max_length": 50})
    ],
    auth_required=True,
    cache_enabled=True,
    pagination=True,
    soft_delete=True
)
# Guardar configuración para uso posterior
config_path = Path("resource_configs") 
config_path.mkdir(exist_ok=True)
with open(config_path / "incident_config.yaml", "w") as f:
    yaml.dump(incident_config.dict(), f, default_flow_style=False)
with open(config_path / "user_config.yaml", "w") as f:
    yaml.dump(user_config.dict(), f, default_flow_style=False)



/var/folders/66/wq6lcq314258fc7_x55djd3c0000gn/T/ipykernel_4100/2115009194.py:48: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  yaml.dump(incident_config.dict(), f, default_flow_style=False)
/var/folders/66/wq6lcq314258fc7_x55djd3c0000gn/T/ipykernel_4100/2115009194.py:50: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  yaml.dump(user_config.dict(), f, default_flow_style=False)


In [132]:
# Templates
model_generator_prompt =ChatPromptTemplate.from_messages([
        ("system", """Eres experto en Pydantic y diseño de APIs con FastAPI.
        Genera modelos Pydantic de nivel PRODUCCIÓN con:
        - Validaciones específicas por tipo de campo
        - Docstrings detallados con ejemplos
        - Field constraints apropiados
        - Validators personalizados para lógica de negocio
        - Modelos de entrada, salida y actualización separados
        """),
                ("human", """
        CONFIGURACIÓN:
        Resource: {resource_name}
        Class: {class_name}
        Fields: {fields}
        Features: auth={auth_required}, cache={cache_enabled}, soft_delete={soft_delete}

        Genera modelos Pydantic profesionales con validaciones robustas.
        """)
    ]) 

In [138]:
model_parser = StrOutputParser()

In [139]:
model_generator_chain= model_generator_prompt | model | model_parser

In [140]:
# Preparar input para el pipeline
models_input = {
    "user": {
        "resource_name": user_config.resource_name,
        "class_name": user_config.class_name,
        "fields": [f.dict() for f in user_config.fields],
        "auth_required": user_config.auth_required,
        "cache_enabled": user_config.cache_enabled,
        "soft_delete": user_config.soft_delete
    },
    "incident": {
        "resource_name": incident_config.resource_name,
        "class_name": incident_config.class_name,
        "fields": [f.dict() for f in incident_config.fields],
        "auth_required": incident_config.auth_required,
        "cache_enabled": incident_config.cache_enabled,
        "soft_delete": incident_config.soft_delete
    }
}

/var/folders/66/wq6lcq314258fc7_x55djd3c0000gn/T/ipykernel_4100/668360229.py:6: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  "fields": [f.dict() for f in user_config.fields],
/var/folders/66/wq6lcq314258fc7_x55djd3c0000gn/T/ipykernel_4100/668360229.py:14: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  "fields": [f.dict() for f in incident_config.fields],


In [142]:
result_model_user= model_generator_chain.invoke(models_input['user'])

In [144]:
result_model_incident= model_generator_chain.invoke(models_input['incident'])

In [145]:
models_chain = RunnableParallel({
    "user_model": RunnableLambda(lambda input: model_generator_chain.invoke(input['user'])),
    "incident_model": RunnableLambda(lambda input: model_generator_chain.invoke(input['incident']))
})

In [146]:
# Invoke the chain using keyword expansion to avoid passing a positional dict
# (passing the dict positionally can lead to BaseModel being called with a positional arg)
result_models = models_chain.invoke(models_input)


In [148]:
print(result_models['user_model'])

from __future__ import annotations

from dataclasses import asdict
from datetime import datetime, timezone
from typing import Annotated, Optional
import base64
import hashlib
import re

from pydantic import (
    BaseModel,
    ConfigDict,
    EmailStr,
    Field,
    computed_field,
    field_validator,
    model_validator,
)
from pydantic.types import StringConstraints


# Tipos reutilizables con restricciones explícitas
IdStr = Annotated[str, StringConstraints(min_length=1, max_length=50, strip_whitespace=True)]
EmailLimited = Annotated[EmailStr, StringConstraints(min_length=1, max_length=100, strip_whitespace=True)]
FullnameStr = Annotated[str, StringConstraints(min_length=1, max_length=200, strip_whitespace=True)]
RoleStr = Annotated[
    str,
    StringConstraints(
        min_length=1,
        max_length=50,
        strip_whitespace=True,
        pattern=r"^[a-z0-9._-]+$"  # slug-like: letras/dígitos/punto/guion/guion_bajo
    ),
]
# Hash de contraseña: longitudes típicas bcrypt

## Parte 3: BackEnd

In [157]:
# 2. Generador de routers FastAPI completos
backend_generator = (ChatPromptTemplate.from_messages([
        ("system", """Eres arquitecto senior FastAPI especialista en APIs RESTful.
        Genera routers de PRODUCCIÓN con:
        - Endpoints CRUD completos (GET, POST, PUT, PATCH, DELETE) tanto para user como para incident
        - Paginación, filtrado y ordenamiento
        - Manejo de errores HTTP consistente
        - Documentación OpenAPI rica
        - Middlewares de auth y cache según configuración
        - Logging estructurado
        - Validación de permisos
        """),
                ("human", """
        Los modelos Pydantic son esenciales para la validación y serialización de datos en FastAPI. 
        A continuación se presentan los modelos Pydantic para los recursos de usuario e incidente:
        MODELO PYDANTIC USER:
        {modelo_pydantic_user}
        MODELO PYDANTIC INCIDENT:
        {modelo_pydantic_incident}
        La configuracion de los modelos Pydantic es la siguiente:
        CONFIGURACIÓN USERS:
        {config_users}
        CONFIGURACION INCIDENT:
        {config_incident}
        La configuración de la base de datos es la siguiente:
        BASE DE DATOS:
        {config_database}
        Los requisitos para la generacion de la API son los siguientes:
        REQUISITOS:
        {requisitos}
        La estructura sugerida para el proyecto es la siguiente:
        {estructura}
                 
        Genera router FastAPI de nivel empresarial.
    """)
    ]) 
    | model
    | StrOutputParser()
)


In [158]:
# Del desño de la arquitectira
backend_requisitos = """
Tecnología y librerías:
- Python 3.12, FastAPI, Uvicorn, SQLAlchemy 2.x, Alembic, Pydantic v2, Passlib (argon2 o bcrypt), PyJWT.

Capas y responsabilidades:
- Routers (FastAPI APIRouter): definen endpoints y validaciones de entrada/salida.
- Services: lógica de negocio (reglas, transacciones orquestadas).
- Repositories: acceso a datos con SQLAlchemy ORM/SQL.
- Schemas (Pydantic): DTOs para requests/responses.
- Core: configuración, seguridad, middlewares.

Autenticación y autorización:
- Login con email+password. Hash con Argon2 (recomendado) o bcrypt.
- JWT de acceso (vida corta, Authorization: Bearer) y refresh (httpOnly, Secure cookie).
- RBAC simple: roles [admin, analyst, user]. Dependencias de ruta verifican scopes/roles.

Endpoints principales (ejemplos):
- POST /auth/login
- POST /auth/refresh
- POST /auth/logout
- GET /users (paginado, filtros: email, role)
- POST /users, GET /users/{id}, PUT /users/{id}, DELETE /users/{id}
- GET /incidents (paginado, filtros: from, to, severity, city, bbox opcional)
- POST /incidents, GET /incidents/{id}, PUT /incidents/{id}, DELETE /incidents/{id}
- GET /analytics/incidents/summary?from=YYYY-MM-DD&to=YYYY-MM-DD
- GET /analytics/incidents/by-severity?from=...&to=...
"""

In [159]:
backend_estructura = """
- backend/
  - app/
    - main.py
    - api/routers/{auth.py, users.py, incidents.py, analytics.py}
    - core/{config.py, security.py}
    - models/{user.py, incident.py}
    - schemas/{user.py, incident.py, auth.py}
    - services/{user_service.py, incident_service.py, analytics_service.py}
    - repositories/{user_repo.py, incident_repo.py}
    - db/{session.py}
    - migrations/
  - tests/
  """

In [160]:
basededatos_config = """
Motor: PostgreSQL 16.

Esquema y relaciones (resumen):
- users(id PK, email UNIQUE, hashed_password, full_name, is_active, created_at TIMESTAMPTZ)
- roles(id PK, name UNIQUE)
- user_roles(user_id FK->users, role_id FK->roles, PK compuesta)
- refresh_tokens(id PK, user_id FK->users, token_hash UNIQUE, exp TIMESTAMPTZ, revoked BOOL)
- incidents(id PK, title, description, occurred_at TIMESTAMPTZ, severity INT CHECK 1..5, lat NUMERIC(9,6), lon NUMERIC(9,6), city TEXT, status TEXT, created_by FK->users)

Índices recomendados:
- users(email)
- incidents(occurred_at), incidents(city), incidents(severity)
- Índice compuesto incidents(city, occurred_at DESC)
- Índices en lat/lon si se filtra por bounding box (ej. BRIN para rango de lat/lon en datasets grandes)

Migraciones:
- Alembic con versionado, seeds iniciales (roles, admin).
"""

In [167]:
backend_inputs = {
    "modelo_pydantic_user": result_models['user_model'],
    "modelo_pydantic_incident": result_models['incident_model'],
    "config_users": json.dumps(models_input['user'], indent=2, ensure_ascii=False),
    "config_incident": json.dumps(models_input['incident'], indent=2, ensure_ascii=False),
    "config_database": basededatos_config,
    "requisitos": backend_requisitos,
    "estructura": backend_estructura
}

In [168]:
result_backend= backend_generator.invoke(backend_inputs)

In [169]:
print(result_backend)

A continuación se entregan dos routers FastAPI de producción para user e incident. Incluyen CRUD completo (GET list, GET by id, POST, PUT, PATCH, DELETE), paginación con filtros y orden, manejo de ETag/condicionales, errores HTTP consistentes, RBAC por roles, cache headers opcionales, y logging estructurado. Están listos para integrarse con servicios y repositorios (inyectados vía dependencias).

Archivo: app/api/routers/users.py
from __future__ import annotations

import logging
from typing import Annotated, Optional, Sequence

from fastapi import APIRouter, Depends, HTTPException, Query, Request, Response, status
from pydantic import BaseModel, Field

from app.core.config import get_settings
from app.core.security import CurrentUser, get_current_user, require_roles
from app.db.session import AsyncSession, get_session
from app.schemas.user import UserCreate, UserOut, UserUpdate
from app.services import user_service

logger = logging.getLogger("app.api.users")
router = APIRouter(prefix

## Parte 4: FrontEnd

In [170]:
# 2. Generador de routers FastAPI completos
frontend_generator = (
    ChatPromptTemplate.from_messages([
        ("system", """Eres programdor senior especialista en React con mas de 10 años de experiencia.
        Genera un frontend de PRODUCCIÓN con las siguientes caracteristicas:
        - Rapido y responsivo
        - Lleve a cabo la ineraccion entre el backend y el usario para cunrir un sistema CRUD completo
         tanto para los users como los incident.
        - Trabajar con los endpoints generados en el codigo especificado en BACKEND_ROUTER.
        - Utilizar componenetes de proposito unico
        - Mantener la modularidad del codigo.
        - Documentar el código
        - Utilizar lazy loading
        - Utilizar listas virtualizadas
        - Flujo de datos unidireccional.
        - Usar Typescript
        """),
                ("human", """
        Los routers FastAPI para la intercaccion con el backend son los siguientes:
        BACKEND_ROUTERS:
        {backend_routers}
        Los modelos Pydantic son los siguiente:
        Los modelos Pydantic son esenciales para la validación y serialización de datos en FastAPI. 
        A continuación se presentan los modelos Pydantic para los recursos de usuario e incidente:
        MODELO PYDANTIC USER:
        {modelo_pydantic_user}
        MODELO PYDANTIC INCIDENT:
        {modelo_pydantic_incident}
        La configuracion de los modelos Pydantic es la siguiente:
        CONFIGURACIÓN USERS:
        {config_users}
        CONFIGURACION INCIDENT:
        {config_incident}
        Los requisitos para la generacion de la API son los siguientes:
        REQUISITOS:
        {requisitos}
        La estructura sugerida para el proyecto es la siguiente:
        {estructura}

        Genera frontend en React de nivel empresarial.
    """)
    ])
    | model
    | StrOutputParser()
)

In [171]:
frontend_requisitos = """
Tecnología: React + Vite, TypeScript, React Router, React Query (tanstack), UI library (MUI), axios.

Páginas clave:
- /login: formulario de autenticación.
- /users: listado CRUD con búsqueda y paginación.
- /incidents: listado y detalle; creación/edición.
- /dashboard: tarjetas de KPIs, gráficos de incidentes por día y severidad; mapa simple (Leaflet) opcional.

Estado y servicios:
- React Query para fetching/caching y reintentos.
- Interceptor axios añade Authorization Bearer y maneja refresh token.
- Guarded routes según rol (admin/analyst/user).
"""

In [172]:
frontend_estructura = """
Estructura (resumen):
- src/
  - pages/{Login.tsx, Users.tsx, Incidents.tsx, Dashboard.tsx}
  - components/{UserForm.tsx, IncidentForm.tsx, Charts.tsx}
  - services/{api.ts, users.ts, incidents.ts, auth.ts}
  - hooks/{useAuth.ts}
  - store/{authStore.ts}
  - routes.tsx
"""

In [173]:
frontend_inputs = {
    "backend_routers": result_backend,
    "modelo_pydantic_user": result_models['user_model'],
    "modelo_pydantic_incident": result_models['incident_model'],
    "config_users": json.dumps(models_input['user'], indent=2, ensure_ascii=False),
    "config_incident": json.dumps(models_input['incident'], indent=2, ensure_ascii=False),
    "requisitos": frontend_requisitos,
    "estructura": frontend_estructura
}

In [174]:
result_frontend = frontend_generator.invoke(frontend_inputs)

In [175]:
print(result_frontend)

A continuación se entrega un frontend de producción en React + Vite + TypeScript, con React Router, React Query, MUI, axios, lazy loading, listas virtualizadas, flujo de datos unidireccional, manejo de ETag (If-None-Match/If-Match), paginación y RBAC. El código está modularizado y documentado.

Instrucciones de instalación (resumen):
- Requisitos: Node 18+
- Dependencias:
  - npm i react-router-dom @tanstack/react-query axios @mui/material @mui/icons-material @emotion/react @emotion/styled react-window zustand recharts
  - opcional mapa: npm i leaflet react-leaflet
- Variables de entorno:
  - VITE_API_BASE_URL=https://tu-backend  (ej: http://localhost:8000)
  - Endpoints de auth asumidos: POST /auth/login, POST /auth/refresh. Ajusta en services/auth.ts según tu backend.

Estructura de archivos (resumen):
- src/
  - main.tsx, theme.ts, queryClient.ts
  - routes.tsx
  - store/authStore.ts
  - hooks/useAuth.ts
  - services/{api.ts, auth.ts, users.ts, incidents.ts, types.ts}
  - utils/etag

## Parte 5: Generador de pruebas backend y frontend

In [ ]:
# Generador de tests exhaustivos para el backend
tests_generator_backend = (
    ChatPromptTemplate.from_messages([
        ("system", """Eres QA Lead especialista en testing de APIs.
        Genera suite de tests COMPREHENSIVA con:
        - Tests unitarios para cada endpoint
        - Tests de integración end-to-end
        - Tests de performance básicos
        - Tests de seguridad (auth, validation)
        - Tests de casos borde y error handling
        - Fixtures y mocks apropiados
        - Cobertura de al menos 90%
        """),
                ("human", """
        ROUTER FASTAPI:
        {router_fastapi}

        MODELO PYDANTIC USER:
        {modelo_pydantic_user}
        MODELO PYDANTIC INCIDENT:
        {modelo_pydantic_incident}
                 
        La configuracion de los modelos Pydantic es la siguiente:
        CONFIGURACIÓN USERS:
        {config_users}
        CONFIGURACION INCIDENT:
        {config_incident}
        La configuración de la base de datos es la siguiente:
        BASE DE DATOS:
        {config_database}

        Genera tests pytest de nivel empresarial.
        """)
    ])
    | model
    | StrOutputParser()
)

In [177]:
tests_generator_backend_inputs = {
    "router_fastapi": result_backend,
    "modelo_pydantic_user": result_models['user_model'],
    "modelo_pydantic_incident": result_models['incident_model'],
    "config_users": json.dumps(models_input['user'], indent=2, ensure_ascii=False),
    "config_incident": json.dumps(models_input['incident'], indent=2, ensure_ascii=False),
    "config_database": basededatos_config
}

In [178]:
result_tests_backend= tests_generator_backend.invoke(tests_generator_backend_inputs)

In [179]:
print(result_tests_backend)

A continuación tienes una suite de tests pytest de nivel empresarial para los routers de Users e Incidents. Incluye:

- Tests unitarios por endpoint (200/201, 304, 4xx mapeos, cabeceras ETag/Cache-Control, orden/paginación/filtros)
- Tests de integración end-to-end con servicio en memoria (optimistic locking If-Match/If-None-Match)
- Tests de performance básicos (concurrencia y latencia)
- Tests de seguridad (RBAC: roles admitidos y denegados)
- Tests de validación de entrada (422 por Pydantic)
- Edge cases y error handling (campos de orden no permitidos, 404, 409, 412, etc.)
- Fixtures y mocks para servicios, settings y dependencias
- Cobertura esperada >90% sobre routers

Estructura propuesta de archivos:
- tests/conftest.py
- tests/routers/test_users_router.py
- tests/routers/test_incidents_router.py
- tests/perf/test_performance.py
- tests/schemas/test_models_validation.py

Contenido:

Archivo: tests/conftest.py
```python
import asyncio
import time
from dataclasses import dataclass

In [180]:
# Generador de tests exhaustivos para el frontend
tests_generator_frontend = (
    ChatPromptTemplate.from_messages([
        ("system", """Eres especialista en testing de frontends en React.
        Genera suite de tests COMPREHENSIVA con:
        - Tests unitarios.
        - Tests de integración end-to-end
        - Tests de performance básicos
        - Tests de seguridad (auth, validation)
        - Cobertura de al menos 90%
        """),
                ("human", """
        FRONTEND REACT:
        {frontend_react}
        
        A continuación se presentan los modelos Pydantic para los recursos de usuario e incidente:
        MODELO PYDANTIC USER:
        {modelo_pydantic_user}
        MODELO PYDANTIC INCIDENT:
        {modelo_pydantic_incident}
        
        La configuracion de los modelos Pydantic es la siguiente:
        CONFIGURACIÓN USERS:
        {config_users}
        CONFIGURACION INCIDENT:
        {config_incident}
        
        Genera tests pytest de nivel empresarial.
        """)
    ])
    | model
    | StrOutputParser()
)

In [181]:
tests_generator_frontend_inputs = {
    "frontend_react": result_frontend,
    "modelo_pydantic_user": result_models['user_model'],
    "modelo_pydantic_incident": result_models['incident_model'],
    "config_users": json.dumps(models_input['user'], indent=2, ensure_ascii=False),
    "config_incident": json.dumps(models_input['incident'], indent=2, ensure_ascii=False)
}

In [182]:
result_frontend_tests= tests_generator_frontend.invoke(tests_generator_frontend_inputs)

In [183]:
print(result_frontend_tests)

A continuación tienes una suite de testing de nivel empresarial para tu frontend React (Vite + TS) con:

- Unit tests (Vitest + Testing Library) que cubren componentes, hooks, store y servicios con axios-interceptors (incluye ETag, If-None-Match/If-Match, refresh 401).
- E2E/Integración con pytest + Playwright (RBAC, auth, CRUD, ETag 304/412, paginación, virtualización, performance básico y controles de seguridad).
- Cobertura mínima 90% garantizada con Vitest (thresholds).
- Instrucciones de ejecución y CI.

1) Dependencias de test

- JS/TS (devDependencies):
  - vitest, @vitest/coverage-istanbul, jsdom
  - @testing-library/react, @testing-library/user-event, @testing-library/jest-dom
  - axios-mock-adapter
  - msw (opcional si prefieres handlers sobre axios-mock-adapter)

- Python (para E2E):
  - pytest, pytest-playwright
  - playwright (instalar browsers: playwright install)

2) Configuración de Vitest e Instrumentación de cobertura

Archivo: vitest.config.ts
-----------------------

## Parte 6: Generador de infraestructura (contenedores y base de datos)

In [185]:
class InfrastructureComponents(BaseModel):
    dockerfile_backend: str = Field(description="Dockerfile optimizado para el backend")
    dockerfile_frontend: str = Field( description="Dockerfile optimizado para el frontend")
    migration: str = Field(description="Migración Alembic con índices")
    docker_compose: str = Field(description="Docker-compose para desarrollo", default="")
    deployment_script: str = Field(description="Script de deployment", default="")
    database_init_script: str = Field(description="Script de inicialización de la base de datos", default="")
    container_deployment_script: str = Field(description="Script de deployment de contenedores", default="")

# 4. Generador de infraestructura (Docker, migrations) - corregido
infra_generator = ChatPromptTemplate.from_messages([
    ("system", """Eres DevOps senior especialista en containerización y databases.
    Genera infraestructura de PRODUCCIÓN:
    - Dockerfile multi-stage optimizado
    - Migración Alembic con índices apropiados
    - Docker-compose para desarrollo
    - Scripts de deployment e inicialización de base de datos
    - Scripts de deployment de contenedores.
    """),
        ("human", """
    A continuación se presentan los modelos Pydantic para los recursos de usuario e incidente:
    MODELO PYDANTIC USER:
    {modelo_pydantic_user}
    MODELO PYDANTIC INCIDENT:
    {modelo_pydantic_incident}
        
    La configuracion de los modelos Pydantic es la siguiente:
    CONFIGURACIÓN USERS:
    {config_users}
    CONFIGURACION INCIDENT:
    {config_incident}
        
    FRONTEND REACT:
    {frontend_react}
    BACKEND FASTAPI:
    {backend_fastapi}
    BASE DE DATOS:
    {config_database}
    Genera infraestructura completa para producción.
    """)
]) | model.with_structured_output(InfrastructureComponents)

In [186]:
infra_generator_inputs = {
    "modelo_pydantic_user": result_models['user_model'],
    "modelo_pydantic_incident": result_models['incident_model'],
    "config_users": json.dumps(models_input['user'], indent=2, ensure_ascii=False),
    "config_incident": json.dumps(models_input['incident'], indent=2, ensure_ascii=False),
    "frontend_react": result_frontend,
    "backend_fastapi": result_backend,
    "config_database": basededatos_config
}

In [187]:
result_infra_generator = infra_generator.invoke(infra_generator_inputs)

In [190]:
print(result_infra_generator.database_init_script)

#!/usr/bin/env bash
# db_init.sh - Inicializa base de datos, aplica migraciones y crea usuario admin
# Requisitos: psql, Docker (opcional si usas contenedor de postgres), Alembic en imagen backend
set -euo pipefail

# Variables de conexión
PGHOST="${PGHOST:-localhost}"
PGPORT="${PGPORT:-5432}"
PGUSER="${PGUSER:-postgres}"
PGPASSWORD="${PGPASSWORD:-postgres}"
PGDATABASE="${PGDATABASE:-ops}"

# Semillas admin
ADMIN_ID="${ADMIN_ID:-usr_admin}"
ADMIN_EMAIL="${ADMIN_EMAIL:-admin@example.com}"
ADMIN_FULLNAME="${ADMIN_FULLNAME:-Admin User}"
ADMIN_ROLE="${ADMIN_ROLE:-admin}"
# Usa un hash real bcrypt/argon2/pbkdf2 (NUNCA texto plano)
ADMIN_HASHED_PASSWORD="${ADMIN_HASHED_PASSWORD:-$2b$12$abcdefghijklmnopqrstuvwxzy1234567890ABCDEFGHIJKLMNO12}"

export PGPASSWORD

# Crear DB si no existe y extensiones
psql -h "$PGHOST" -p "$PGPORT" -U "$PGUSER" -v ON_ERROR_STOP=1 <<SQL
DO $$
BEGIN
   IF NOT EXISTS (SELECT FROM pg_database WHERE datname = '$PGDATABASE') THEN
      PERFORM dblink_exec('dbname=' ||

## Parte 7: Orquestación de componentes
Se crean 4 generadores:
- model_pydantic_user: produce los modelos pydantic para la validadción de los datos del usuario
- model_pydantic_user: produce los modelos pydantic para la validadción de los datos de incidente
- backend_generator: crea el backend con FastAPI para el CRUD completo
- frontend_generator: crea el frontend de acuerdo a la arquitectura obtenida en la fase previa 
- tests_backend_generator: crea los pruebas para el backend
- tests_frontend_generator: crear las pruebas para el frontend 
- infra_generator: crea los archivos para generar la infraestructura para el deployment (dockerfiles, migraciones, base de datos)

Se va a tomar como base la arquitectura obtenida en la fase anterior

In [191]:
# Clase para concentrar las salidas de las pipelines

class GeneratedComponents(BaseModel):
    modelo_pydantic_user: str = Field(description="Modelo Pydantic con validaciones avanzadas para usuarios")
    modelo_pydantic_incident: str = Field(description="Modelo Pydantic con validaciones avanzadas para incidentes")
    backend_fastapi: str = Field(description="Router FastAPI con CRUD completo")
    frontend_react: str = Field(description="Frontend React")
    tests_pytest: str = Field(description="Suite de tests exhaustiva")
    tests_react: str = Field(description="Suite de tests exhaustiva para React")
    dockerfile_backend: str = Field(description="Dockerfile optimizado para el backend")
    dockerfile_frontend: str = Field(description="Dockerfile optimizado para el frontend")
    docker_compose: str = Field(description="Docker-compose para desarrollo")
    deployment_script: str = Field(description="Script de deployment")
    database_init_script: str = Field(description="Script de inicialización de la base de datos")
    container_deployment_script: str = Field(description="Script de deployment de contenedores")
    alembic_migration: str = Field(description="Migración Alembic para base de datos")

In [192]:
advanced_crud_inputs = {
    "user": models_input['user'],
    "incident": models_input['incident'],
    "basededatos_config": basededatos_config,
    "backend_requisitos": backend_requisitos,
    "backend_estructura": backend_estructura,
    "frontend_requisitos": frontend_requisitos,
    "frontend_estructura": frontend_estructura
}

In [200]:
# Definimos la pipeline completa de generación avanzada
def advanced_crud_pipeline(base_input: Dict[str, Any]) -> GeneratedComponents:
    # Creamos los modelos Pydantic en paralelo
    models_output = RunnableParallel({
        "user": RunnableLambda(lambda input: model_generator_chain.invoke(input['user'])),
        "incident": RunnableLambda(lambda input: model_generator_chain.invoke(input['incident']))
    }).invoke(base_input)
    # Inicializamos las variables del backend
    backend_inputs_aux = {
    "modelo_pydantic_user": models_output['user'],
    "modelo_pydantic_incident": models_output['incident'],
    "config_users": json.dumps(base_input['user'], indent=2, ensure_ascii=False),
    "config_incident": json.dumps(base_input['incident'], indent=2, ensure_ascii=False),
    "config_database": base_input['basededatos_config'],
    "requisitos": base_input['backend_requisitos'],
    "estructura": base_input['backend_estructura']
    }
    # Generamos el backend
    backend_output = backend_generator.invoke(backend_inputs_aux)
    # Inicializamos las variables del frontend
    frontend_inputs_aux = {
        "backend_routers": backend_output,
        "modelo_pydantic_user": models_output['user'],
        "modelo_pydantic_incident": models_output['incident'],
        "config_users": json.dumps(base_input['user'], indent=2, ensure_ascii=False),
        "config_incident": json.dumps(base_input['incident'], indent=2, ensure_ascii=False),
        "requisitos": base_input['frontend_requisitos'],
        "estructura": base_input['frontend_estructura']
    }
    # Generamos el frontend
    frontend_output = frontend_generator.invoke(frontend_inputs_aux)
    # Generamos los tests y la infraestructura en paralelo
    tests_infra_output = RunnableParallel({
        "backend": RunnableLambda(lambda input: tests_generator_backend.invoke({
            "router_fastapi": backend_output,
            "modelo_pydantic_user": models_output['user'],
            "modelo_pydantic_incident": models_output['incident'],
            "config_users": json.dumps(base_input['user'], indent=2, ensure_ascii=False),
            "config_incident": json.dumps(base_input['incident'], indent=2, ensure_ascii=False),
            "config_database": base_input['basededatos_config']
        })),
        "frontend": RunnableLambda(lambda input: tests_generator_frontend.invoke({
            "frontend_react": frontend_output,
            "modelo_pydantic_user": models_output['user'],
            "modelo_pydantic_incident": models_output['incident'],
            "config_users": json.dumps(base_input['user'], indent=2, ensure_ascii=False),
            "config_incident": json.dumps(base_input['incident'], indent=2, ensure_ascii=False)
        })),
        "infra": RunnableLambda(lambda input: infra_generator.invoke({
            "modelo_pydantic_user": models_output['user'],
            "modelo_pydantic_incident": models_output['incident'],
            "config_users": json.dumps(base_input['user'], indent=2, ensure_ascii=False),
            "config_incident": json.dumps(base_input['incident'], indent=2, ensure_ascii=False),
            "frontend_react": frontend_output,
            "backend_fastapi": backend_output,
            "config_database": base_input['basededatos_config']
        }))
    }).invoke(base_input)
    # Combinamos todos los outputs en la clase GeneratedComponents
    return GeneratedComponents(
        modelo_pydantic_user=models_output['user'],
        modelo_pydantic_incident=models_output['incident'],
        backend_fastapi=backend_output,
        frontend_react=frontend_output,
        tests_pytest=tests_infra_output['backend'],
        tests_react=tests_infra_output['frontend'],
        dockerfile_backend=tests_infra_output['infra'].dockerfile_backend if tests_infra_output['infra'] else "",
        dockerfile_frontend=tests_infra_output['infra'].dockerfile_frontend if tests_infra_output['infra'] else "",
        docker_compose=tests_infra_output['infra'].docker_compose if tests_infra_output['infra'] else "",
        deployment_script=tests_infra_output['infra'].deployment_script if tests_infra_output['infra'] else "",
        database_init_script=tests_infra_output['infra'].database_init_script if tests_infra_output['infra'] else "",
        container_deployment_script=tests_infra_output['infra'].container_deployment_script if tests_infra_output['infra'] else "",
        alembic_migration=tests_infra_output['infra'].migration if tests_infra_output['infra'] else ""
    )
     
    

In [ ]:
# Ejecutamos la pipeline completa de generación avanzada
generated_components = advanced_crud_pipeline(advanced_crud_inputs)